# 7章 ロジスティクスネットワークの最適設計を行う10本ノック

ここでは、最適化計算を行ういくつかのライブラリを用いて、最適化計算を実際に行っていきます。  
そして、前章で用いたネットワーク可視化などの技術を駆使し、計算結果の妥当性を確認する方法についても学んでいきます。

### ノック６１：輸送最適化問題を解いてみよう

In [5]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

df_tc = pd.read_csv('trans_cost.csv', index_col='工場')
print(df_tc)
df_demand = pd.read_csv('demand.csv')
df_supply = pd.read_csv('supply.csv')

np.random.seed(1)
nw =  len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))

m1 = model_min()
v1  = {(i, j):LpVariable('v%d_%d'%(i, j), lowBound=0) for i, j in pr}

m1 += lpSum(df_tc.iloc[i][j] * v1[i,j] for i, j in pr)
for i in range(nw):
    m1 += lpSum(v1[i, j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i,j] for i in range(nw)) >= df_demand.iloc[0][j]
m1.solve()

df_tr_sol = df_tc.copy()
total_cost = 0
for k, x in v1.items():
    i, j = k[0], k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j] * value(x)

print(df_tr_sol)
print('総輸送コスト: ' +str(total_cost))

    F1  F2  F3  F4
工場                
W1  10  10  11  27
W2  18  21  12  14
W3  15  12  14  12
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kazuyaaoki/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/909f2d3abef74a59bc3509360d0a02a3-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/909f2d3abef74a59bc3509360d0a02a3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 49 RHS
At line 57 BOUNDS
At line 58 ENDATA
Problem MODEL has 7 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 113 (4)
7  Obj 1296
Optimal - objective value 1296
Optimal objective 1296 - 7 iterations time 0.002
Option for printingOptio

### ノック６２：最適輸送ルートをネットワークで確認しよう

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

df_tr = df_tr_sol.copy()
df_pos = pd.read_csv('trans_route_pos.csv')
print(df_pos)

G = nx.Graph()

for i in range(len(df_pos.columns)):
    G_add_node(df_pos.columns[i])
    
num_pre = 0
edge_weights = []
size = 0.1
for i in range(len(df_pos.columns)):
    for j in range(lne(df_pos.columns)):
        if not(i==j):
            G.add_edge(df_pos.columns[i], df_pos.columns[j])
            if num_pre < len(G.edge):
                num_pre = len(G.edge)
                weight  = 0
                if (df_pos.columns[i] in df_tr.columns) and (df_pos.columns[j] in df_tr.index):
                    if df_tr[df_pos.columns[i]][df_pos.columns[j]]:
                        weight = df_tr[df_pos.columns[i]][df_pos.columns[j]] * size
                elif(df_pos.columns[j] in df_tr.columns) and (df_pos.columns[i] in df_tr.index):
                    if df_tr[df_pos.columns[j]][df_pos.columns[i]]:
                        weight = df_tr[df_pos.columns[j]][df_pos.columns[i]] *size
                edge_weights.append(weight)

pos = {}
for i in range(len(df_pos.columns)):
    node = df_pos.columns[i]
    pos[node] = (df_pos[node][0], df_pos[node][1])
    
    
nx.draw(G, pos, with_labels = True, font_size=16, node_size=1000, node_color = 'k', font_color=0)

### ノック６３：最適輸送ルートが制約条件内に収まっているかどうかを確認しよう

### ノック６４：生産計画に関するデータを読み込んでみよう

### ノック６５：利益を計算する関数を作ってみよう

### ノック６６：生産最適化問題を解いてみよう

### ノック６７：最適生産計画が制約条件内に収まっているかどうかを確認しよう

### ノック６８：ロジスティクスネットワーク設計問題を解いてみよう

### ノック６９：最適ネットワークにおける輸送コストとその内訳を計算しよう

### ノック７０：最適ネットワークにおける生産コストとその内訳を計算しよう